In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
from fbprophet import Prophet

In [2]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

HOSTED_APP_URL = 'https://iotproject.eu.pythonanywhere.com'
API_PLANT_DATA = '/api/v1/plant_data'

df_orig = pd.read_json(HOSTED_APP_URL + API_PLANT_DATA)
df_orig.head(5)

,bridge_id,creation_date,dht_humidity,dht_temperature,gateway_id,humidity_1,humidity_2,humidity_3,id,last_modified,luminosity_1,luminosity_2,plant_id,plant_type_id,temperature,timestamp
0,RASPBERRY001,2021-04-02T00:29:21,76,23,ARDUINO001,NaN,NaN,NaN,1,2021-04-01T22:29:21,12,11,c5a3b03d82944886bb1fdb30612d5c5d,1,24.47,1200549
1,RASPBERRY001,2021-04-02T00:39:22,80,23,ARDUINO001,NaN,NaN,NaN,2,2021-04-01T22:39:22,12,11,c5a3b03d82944886bb1fdb30612d5c5d,1,24.47,1800823
2,RASPBERRY001,2021-04-02T00:59:19,81,23,ARDUINO001,NaN,NaN,NaN,3,2021-04-01T22:59:20,13,11,c5a3b03d82944886bb1fdb30612d5c5d,1,24.29,3001370
3,RASPBERRY001,2021-04-01T22:46:47,71,24,ARDUINO001,NaN,NaN,NaN,4,2021-04-01T23:09:21,62,65,c5a3b03d82944886bb1fdb30612d5c5d,1,24.55,600275
4,RASPBERRY001,2021-04-01T22:56:43,74,23,ARDUINO001,NaN,NaN,NaN,5,2021-04-01T23:09:21,63,66,c5a3b03d82944886bb1fdb30612d5c5d,1,24.38,1200548


In [3]:
df_orig['luminosity_mean'] = df_orig[['luminosity_1', 'luminosity_2']].mean(axis=1)
df_orig['luminosity_mean'].head(5)


0    11.5
1    11.5
2    12.0
3    63.5
4    64.5
Name: luminosity_mean, dtype: float64

In [4]:
df_orig['humidity_mean'] = df_orig[['humidity_1', 'humidity_2', 'humidity_3']].mean(axis=1)
df_orig['humidity_median'] = df_orig[['humidity_1', 'humidity_2', 'humidity_3']].median(axis=1)
df_orig['creation_datetime'] = pd.to_datetime(df_orig['creation_date'])

In [5]:
# keep from 10th may to 1st aug 2021

df_trimmed = df_orig[(df_orig['creation_datetime'] > '2021-05-09') & (df_orig['creation_datetime'] < '2021-08-03')]
# df_trimmed.head(5)

In [6]:
from hampel import hampel
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
import itertools

In [7]:
df_trimmed = df_trimmed.reset_index(drop=True)

In [8]:
df = pd.DataFrame(columns=['ds', 'y', 'luminosity', 'humidity'])
df['ds'] = pd.to_datetime(df_trimmed['creation_date'])
df['luminosity'] = hampel(df_trimmed['luminosity_mean'], window_size=5, n=3, imputation=True)
df['humidity'] = hampel(df_trimmed['humidity_median'], window_size=5, n=3, imputation=True)

In [9]:
df['y'] = df['luminosity']

In [12]:
m = Prophet()
m.fit(df)
periods = 2*24*60//10
future = m.make_future_dataframe(periods=periods, freq='10min')
forecast = m.predict(future)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [13]:
def create_param_combinations(**param_dict):
    param_iter = itertools.product(*param_dict.values())
    params =[]
    for param in param_iter:
        params.append(param)
    params_df = pd.DataFrame(params, columns=list(param_dict.keys()))
    return params_df

def single_cv_run(history_df, metrics, param_dict):
    mdl = Prophet(**param_dict)
    mdl.fit(history_df)
    df_cvl = cross_validation(mdl, horizon='48 hours')
    df_perf = performance_metrics(df_cvl).mean().to_frame().T
    df_perf['params'] = str(param_dict)
    df_perf = df_perf.loc[:, metrics]
    return df_perf

In [14]:
# The best param combination (found earlier in 2021) was
# {'changepoint_prior_scale': 0.005,
# 'changepoint_range': 0.9,
# 'seasonality_prior_scale': 0.1,
# 'seasonality_mode': 'additive'}

param_grid = {
                'changepoint_prior_scale': [0.001, 0.005, 0.010],
                'changepoint_range': [0.85, 0.9, 0.95],
                'seasonality_prior_scale':[0.05, 0.1, 0.5],
                'seasonality_mode': ['additive'],
              }
metrics = ['horizon', 'rmse', 'mape', 'params']
results = []
params_df = create_param_combinations(**param_grid)

In [15]:
# set var to True if you want to find hyper-params
find_hyper_params = False
if find_hyper_params:
    for param in params_df.values:
        param_dict = dict(zip(params_df.keys(), param))
        cv_df = single_cv_run(df,  metrics, param_dict)
        results.append(cv_df)
    results_df = pd.concat(results).reset_index(drop=True)
    best_param = results_df.loc[results_df['mape'] == min(results_df['mape']), ['params']]
    print(f'\n The best param combination is {best_param.values[0][0]}')
    results_df.to_csv('results_hyperparam_final_cv.csv')
    best_param.to_csv('best_results_final.csv')

In [10]:
best_hyperparams = {
'changepoint_prior_scale': 0.001,
'changepoint_range': 0.9,
'seasonality_prior_scale': 0.5,
'seasonality_mode': 'additive'
}

In [12]:
best_model = Prophet(**best_hyperparams)
best_model.fit(df)
periods = 2*24*60//10
future = best_model.make_future_dataframe(periods=periods, freq='10min')
forecast = best_model.predict(future)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [ ]:
fig1 = best_model.plot(forecast)
fig2 = best_model.plot_components(forecast)